<h1>Transfer learning for regression under differential privacy</h1>

<h1>生成数据训练模型</h1>

<h1>导入库</h1>

In [30]:
import numpy as np 
import random

<h1>求解Lasso</h1>

In [31]:
# encoding: utf-8

alpha = 0.0001
Lambda = 0.1
# 精度设定
epsilon = 1e-8
#api
def norm1(w,Lambda):
    cnt=[]
    for i in w:
        cnt.append(Lambda*abs(i))
    return cnt

def add_laplace_noise(u,b,n):
    laplace_noise = np.random.laplace(u,b,n) # 产生Laplace分布的噪声
    return laplace_noise

#https://blog.csdn.net/weixin_35757704/article/details/109147968
# 采用梯度下降法求解一元线性回归
def solve_by_gradient(x,y):
    # 获取循环的长度
    m = len(x)
    a, b,= [0 for i in range(len(x[0]))],0
    for i in range(50):
        grad_a, grad_b = [0 for i in range(len(x[0]))], 0
        grad_a+=add_laplace_noise(0,sum(a)/len(a),len(grad_a))
        for i in range(m):
            # 求（a*x(i)+b-y[i])的a,b偏导
            # 这里使用common感觉更好,公式相同处
            common=0
            for j in range(len(x[i])):
                common+= a[j] * x[i][j]
            common+=b
            common-=y[i]
            common*=2
            #print(common)
            for j in range(len(x[i])):
                grad_a[j]+=x[i][j] * common
            grad_a+=norm1(a,Lambda)
            grad_b += common
            #print("gr1",grad_a,grad_b)
            for j in range(len(grad_a)):
                grad_a[j]=grad_a[j]/m
            grad_b = grad_b / m
            #print("gr2",grad_a,grad_b)
            # 梯形下降(梯形负方向,速度下降最快)迭代求符合最小值的a,b
            # alpha设置迭代步长,即学习率
            for j in range(len(x[i])):
                a[j]-=alpha*grad_a[j]
            b -= alpha * grad_b

            sse = 0
            #print(a,b)
            for j in range(m):
                sum_xw=0
                for k in range(len(x[j])):
                    sum_xw+=a[k]*x[j][k]
                sse += (sum_xw + b - y[j]) ** 2 / m
            sse+=sum(norm1(a,Lambda))

            # 拟合结果判断相差绝对值
            if sse < epsilon:
                break
            else:
                sse2 = sse
    #print('{0} * x + {1}'.format(a, b))
    #print(sse)
    return sse


<h1>模拟实验</h1>

In [32]:
x = [[1.,1.],[2.,2.], [3.,3.], [4.,4.],[5.,5.]]
y = [3., 5., 7., 9., 11.]

In [33]:
#未迁移
# 目标函数y=2x+1
sumx=0
for i in range(100):
    sumx+=solve_by_gradient(x,y)
print(sumx/100)

34.17569529149986


In [34]:
#迁移五个
for i in range(6,11):
    x.append([float(i),float(i)])
for i in range(6,11):
    y.append(float(2*i+1))


sumx=0.0
for i in range(100):
    sumx+=solve_by_gradient(x,y)
print(sumx/100)

33.12829401014668


In [35]:
#迁移十个
for i in range(11,16):
    x.append([float(i),float(i)])
for i in range(11,16):
    y.append(float(2*i+1))


sumx=0.0
for i in range(100):
    sumx+=solve_by_gradient(x,y)
print(sumx/100)

11.112408338780718


In [36]:
#迁移十五个
for i in range(16,21):
    x.append([float(i),float(i)])
for i in range(16,21):
    y.append(float(2*i+1))


sumx=0.0
for i in range(100):
    sumx+=solve_by_gradient(x,y)
print(sumx/100)

1.8909373370345326


In [37]:
#迁移二十个
for i in range(21,26):
    x.append([float(i),float(i)])
for i in range(21,26):
    y.append(float(2*i+1))


sumx=0.0
for i in range(100):
    sumx+=solve_by_gradient(x,y)
print(sumx/100)

0.5093407044005585
